# Forcing 

In [43]:
import xarray as xr
import csv
import pandas as pd
import pickle
import numpy as np
import os
import datetime 
import sys
import matplotlib.pyplot as plt
import netCDF4 as nc
import os.path
import webbrowser
import time
from scipy.io import netcdf
import netCDF4
from datetime import date, timedelta, datetime
import globe


In [200]:
## Read in the data
filename_precip='/Users/jacoposala/Downloads/cfsv2-sea_2015_01hr_precip.nc' # precipitation rate
filename_windspd='/Users/jacoposala/Downloads/cfsv2-sec2_2015_01hr_wndspd.nc'

ds_precip=xr.open_dataset(filename_precip,decode_times=False)
ds_windspd=xr.open_dataset(filename_windspd,decode_times=False)

varname_1 = 'wndspd' # wind
varname_1_value = 32.9244 # wind speed threshold [m/s], corresponds to 64 knots
varname_2 = 'precip' # precipitation

In [204]:
# At each lat/lon, keep the max wind speed event (above the WS threshold)
# then remove all the other events within +- 5 days
# find the second-max WS event and store it
# remove all the other events within +- 5 days
# and continue until no other events above the WS tresholds need to be stored at the location being considered,
# then move on to the next lat/lon

column_names = ['lat','lon','time','wind_speed','precipitation','ID']
df_all = pd.DataFrame(columns = column_names)
delta_time = 120 # hours in 5 day interval
event_ID = 1 # ID for events

start = datetime(1900,12,31,0,0,0) # This is the "days since" part

for current_lat in ds_windspd.Latitude.to_numpy(): # Loop through lat
    # Is this lat in the tropics?
    if (current_lat > -30) & (current_lat < 30):
        for current_lon in ds_windspd.Longitude.to_numpy(): # Loop through lon
            # Is this lat/lon in the ocean?
            if globe.is_ocean(current_lat, (current_lon + 180) % 360 - 180):

                # wind
                d_1 = ds_windspd.sel(Latitude = current_lat, Longitude = current_lon)
                # precipitation 
                d_2 = ds_precip.sel(Latitude = current_lat, Longitude = current_lon)
    
                # Ignore this lat/lon combination if there are no hurricanes
                if np.sum(d_1[varname_1] >= varname_1_value).values > 0: 
                    # Select only time stamps at this location with WS > 64 knots
                    d_1_masked = d_1[varname_1].where(d_1[varname_1] >= varname_1_value, drop=True) 
                    d_2_masked = d_2[varname_2].where(d_1[varname_1] >= varname_1_value, drop=True)

                    d_1_stacked = d_1_masked.stack(x=['MT'])
                    d_2_stacked = d_2_masked.stack(x=['MT'])

                    d_1_list = d_1_stacked[d_1_stacked.notnull()]
                    d_2_list = d_2_stacked[d_2_stacked.notnull()]
                    
                    # Convert to DataFrame
                    d_1_list_df = d_1_list.to_dataframe()
                    d_1_list_df['MT'] = np.array(d_1_list.MT)
                    
                    # Loop through selected time stamps at this lat/lon
                    while len(d_1_list_df) > 0: # continue until there are any events left
                        # Find index of max WS in the selected time stamps at this lat/lon
                        i_max_d1 = max(range(len(d_1_list_df)), key=d_1_list.__getitem__)
                        # Store data for the time stamp with max wind speed among those left
                        temp = pd.DataFrame(
                                {
                                    # Lat
                                    'lat': d_1_list[i_max_d1].Latitude.values,
                                    # Lon
                                    'lon': d_1_list[i_max_d1].Longitude.values,
                                    # Time
                                    'time': [start + timedelta(float(d_1_list.MT[i_max_d1].values))],
                                    # WS
                                    'wind_speed': [d_1_list[i_max_d1].values],
                                    # Precipitation
                                    'precipitation': [d_2_list[i_max_d1].values],
                                    # ID
                                    'ID': event_ID
                                })

                        # Add event to final DataFrame
                        df_all = pd.concat([df_all, temp])
                        
                        # Increase event ID
                        event_ID = event_ID + 1
        
                        # Remove from list all the events within +- 5 days from stored event
                        time_recorded = float(d_1_list.MT[i_max_d1].values) # in days since 1900
                        d_1_list_df.drop(d_1_list_df[(d_1_list_df.MT > time_recorded -5) & (d_1_list_df.MT < time_recorded + 5)].index, inplace=True)

            
# Show final result 
df_all = df_all.reset_index(drop = True)
df_all

KeyboardInterrupt: 

In [ ]:
# Save
df_all.to_csv('df_forcing_2015_newapproach.csv')

In [207]:
d_1_list_df

,Date,Latitude,Longitude,wndspd,MT
MT,,,,,
41739.875000,2.015041e+07,-24.838156,68.931818,34.408203,41739.875000
41739.916667,2.015041e+07,-24.838156,68.931818,34.414062,41739.916667
41739.958333,2.015041e+07,-24.838156,68.931818,33.219727,41739.958333
41740.000000,2.015041e+07,-24.838156,68.931818,33.414062,41740.000000


In [36]:
column_names = ['lat','lon','time','wind_speed','precipitation']
df_all = pd.DataFrame(columns = column_names)
delta_time = 120 # hours in 5 day interval

start = datetime(1900,12,31,0,0,0) # This is the "days since" part

for i in ds_windspd.MT.to_numpy(): # Loop through timesteps
    # wind
    d_1 = ds_windspd.sel(MT=i)
    # precipitation 
    d_2 = ds_precip.sel(MT=i)
    
    # Select only time stamps and locations with WS > 64 knots
    if np.sum(d_1[varname_1] >= varname_1_value).values > 0: # ignore this time stamp if there are no hurricanes
        d_1_masked = d_1[varname_1].where(d_1[varname_1] >= varname_1_value, drop=True) 
        d_2_masked = d_2[varname_2].where(d_1[varname_1] >= varname_1_value, drop=True)

        d_1_stacked = d_1_masked.stack(x=['Latitude','Longitude'])
        d_2_stacked = d_2_masked.stack(x=['Latitude','Longitude'])

        d_1_list = d_1_stacked[d_1_stacked.notnull()]
        d_2_list = d_2_stacked[d_2_stacked.notnull()]
        
        # Populate DataFrame
        for j in np.arange(d_1_list.shape[0]):
            # If within tropics
            if (d_1_list.Latitude[j].values > -30) & (d_1_list.Latitude[j].values < +30):
                # If no rows with same lat, lon and within +-5 days are present:
                if df_all[(df_all.lat == d_1_list.Latitude[j].values) & (df_all.lon == d_1_list.Longitude[j].values) \
                   & (df_all.time > start + timedelta(i) - timedelta(hours=delta_time)) & \
                   (df_all.time < start + timedelta(i) + timedelta(hours=delta_time))].empty:
                    temp = pd.DataFrame(
                            {
                                # Lat
                                'lat': d_1_list.Latitude[j].values,
                                # Lon
                                'lon': d_1_list.Longitude[j].values,
                                # Time
                                'time': [start + timedelta(i)],
                                # WS
                                'wind_speed': [d_1_list[j].values],
                                # Precipitation
                                'precipitation': [d_2_list[j].values]
                            })
                else:
                    # if there already something stored in the same spatio-temporal vicinity, check the winds and keep the stronger one
                    WS_already_stored = df_all[(df_all.lat == d_1_list.Latitude[j].values) & (df_all.lon == d_1_list.Longitude[j].values) \
                                               & (df_all.time > start + timedelta(i) - timedelta(hours=delta_time)) & \
                                                (df_all.time < start + timedelta(i) + timedelta(hours=delta_time))].wind_speed
                    if float(d_1_list[j].values) > float(WS_already_stored):
                        # Remove weaker row
                        df_all[(df_all.lat == d_1_list.Latitude[j].values) & (df_all.lon == d_1_list.Longitude[j].values) \
                                               & (df_all.time > start + timedelta(i) - timedelta(hours=delta_time)) & \
                                                (df_all.time < start + timedelta(i) + timedelta(hours=delta_time))].drop()

                        # Use the stronger wind instead
                        temp = pd.DataFrame(
                                {
                                    # Lat
                                    'lat': d_1_list.Latitude[j].values,
                                    # Lon
                                    'lon': d_1_list.Longitude[j].values,
                                    # Time
                                    'time': [start + timedelta(i)],
                                    # WS
                                    'wind_speed': [d_1_list[j].values],
                                    # Precipitation
                                    'precipitation': [d_2_list[j].values]
                                })                           


                df_all = pd.concat([df_all, temp])

# Show final result 
df_all = df_all#.reset_index()


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [41]:
float(d_1_list[j].values)

33.2958984375

In [40]:
float(WS_already_stored)

33.6240234375

In [ ]:
# Save
df_all.to_csv('df_forcing_2015.csv')

In [ ]:
# df_all[(df_all.lat > -30) & (df_all.lon < 30)]


In [26]:
df_all = df_all.reset_index(drop=True)

In [27]:
df_all

,lat,lon,time,wind_speed,precipitation
0,-18.296419,58.704545,2015-01-13 13:00:00,33.624023,0.0014979057
1,-18.296419,58.909091,2015-01-13 13:00:00,33.243164,0.0016555786
2,-18.09199,58.704545,2015-01-13 13:00:00,33.341797,0.0008672066
3,-18.09199,58.909091,2015-01-13 13:00:00,33.948242,0.0011571236
4,-18.09199,59.113636,2015-01-13 13:00:00,33.13379,0.0014724731
...,...,...,...,...,...
9479,-25.655873,76.499999,2015-01-18 09:00:00,33.176758,0.001209259
9480,-27.086878,79.772727,2015-01-18 15:00:00,33.0166,0.00012207031
9481,-27.495736,81.818181,2015-01-18 19:00:00,33.216797,0.00024541095
9482,-27.291307,81.613636,2015-01-18 19:00:00,33.32129,0.000118255615


In [28]:
column_names = ['lat','lon','time','ID']
event_ID = 1
delta_time = 120
df_all_new  = pd.DataFrame(columns = column_names)

for i in np.arange(0,len(df_all)): # loop through rows of original dataframe
    # Do not consider this df_all all if df_all_new already includes something at the same lat, lon 
    # and within +- 5 days of it
    if df_all_new[(df_all_new.lat == df_all.lat[i]) & (df_all_new.lon == df_all.lon[i]) \
           & (df_all_new.time > df_all.time[i] - timedelta(hours=delta_time)) & \
           (df_all_new.time < df_all.time[i] + timedelta(hours=delta_time))].empty:

        # Create a DataFrame with lat, lon, time
        temp_all = pd.DataFrame(columns = column_names)
        # Populate DataFrame with same lat, lon, and all hourly timestamps within +- days
        for j in np.arange(-delta_time, delta_time): # loop across +- 5 days
            temp = pd.DataFrame(
                            {
                                # Lat
                                'lat': df_all.lat[i],
                                # Lon
                                'lon': df_all.lon[i],
                                # Time
                                'time': [df_all.time[i] + timedelta(hours=int(j))],
                                # Event ID
                                'ID': event_ID
                            })
            temp_all = pd.concat([temp_all, temp]).reset_index(drop=True)

        # Concatenate to final DataFrame
        df_all_new = pd.concat([df_all_new, temp_all]).reset_index(drop=True)
        # Increase index to keep track of events
        event_ID = event_ID + 1

KeyboardInterrupt: 

In [29]:
df_all_new

,lat,lon,time,ID
0,-18.296419,58.704545,2015-01-08 13:00:00,1
1,-18.296419,58.704545,2015-01-08 14:00:00,1
2,-18.296419,58.704545,2015-01-08 15:00:00,1
3,-18.296419,58.704545,2015-01-08 16:00:00,1
4,-18.296419,58.704545,2015-01-08 17:00:00,1
...,...,...,...,...
186475,-19.114136,66.477272,2015-01-20 23:00:00,777
186476,-19.114136,66.477272,2015-01-21 00:00:00,777
186477,-19.114136,66.477272,2015-01-21 01:00:00,777
186478,-19.114136,66.477272,2015-01-21 02:00:00,777


In [30]:
i

5415

In [ ]:
# Save
df_all.to_csv('df_forcing_2015.csv')